<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/generate_synthetic_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries

In [1]:
!pip install transformers
!pip install sentence-transformers
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 10.5 MB/s 
     |████████████████████████████████| 6.6 MB 30.9 MB/s 
     |████████████████████████████████| 120 kB 54.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 1.3 MB 17.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4d33e6bf9f6ab791a627da13842e05c8dc7c24c7c7b8300701c3ee3b078bde61
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 10.4 MB/s 
     |██████████████████████

In [2]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import re

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict
from datasets import Dataset

import numpy as np
from numpy import random
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# Load dataset

In [3]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

In [4]:
!ls

drive  emp_chatbot  sample_data


In [5]:
os.chdir('/content/emp_chatbot/data')

In [6]:
!gdown --id 1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
To: /content/emp_chatbot/data/all_emotions_responses.json
100% 1.24M/1.24M [00:00<00:00, 112MB/s]


In [7]:
data_files = {"train": "all_emotions_responses.json"}
ds = load_dataset('json', data_files=data_files)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e8c4d2e6d2810eed/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
ds.reset_format()

In [9]:
import pandas as pd
ds.set_format(type="pandas")

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?'],
        num_rows: 200
    })
})

# Generate utterances


## Define split utterance dictionary with keys as initial questions and additional questions.

In [11]:
split_utterances = dict()
columns = ds['train'].column_names
columns_indices = list(range(9,len(columns)-1))
additional_questions_columns = [columns[index] for index in columns_indices]
joy_question_column = [columns[16]]
initial_questions_indices = [4,5,6,7,8]
initial_questions_columns = [columns[index] for index in initial_questions_indices]

In [12]:
joy_question_column

['Would you like to attempt protocols for joy?']

In [13]:
# check columns identified correctly
print(initial_questions_columns)
print(joy_question_column)
print(additional_questions_columns)

['Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?']
['Would you like to attempt protocols for joy?']
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']


In [14]:
# define base questions in russian to be used in a splitting operation
initial_base_questions = ["Вы полагаете, что ваше состояние связано с каким-то конкретным неприятным опытом / событием?", "Эта проблема возникла у вас?", "Методика 6 вызвала у Вас какие-то плохие чувства дискомфорта?", "Методика 11 вызвала у Вас какие-то плохие чувства дискомфорта?", "я задам вам несколько дополнителных вопросов - вы не против?"]
joy_question = ['Я бы хотел предложить Вам одно упражнение']
base_questions = ["вы испытываете к кому-либо злость, зависть или агрессию?", "Вы чуствуете ответственность за кого-либо?", "Вам кажется, что вы хотите кого то контролировать?", "Вы испытываете ощущение жертвы и вините кого то в своём состоянии?", "Считаете ли вы себя причиной проблемы?", "Как часто Вы настаиваете на своей позиции не принимая во внимание позицию собеседников?", "Есть ли у вас сложности с друзьями, возможно в личной жизни?"]

In [15]:
split_utterances["base_questions"] = base_questions
split_utterances ['initial_base_questions'] = initial_base_questions
split_utterances['joy_question'] = joy_question

## Split original utterances into empathy and base utterance


In [16]:
# extract emotion names
ds.set_format(type="pandas")
emotions = ds['train'].unique('emotion')
ds.reset_format()

In [17]:
# define a tokenizer for feature extraction
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

In [18]:
# a function to split an utterance into a base and an empathetic part
def split_emp_base(corpus, base_utterance):
  base_embedding = pipe(base_utterance)
  base_utterances = []
  empathy_utterances = []

  for sentence in corpus:
    my_list = re.split('[.!?]', sentence)
    for sentence in my_list:
      if len(sentence) <= 5:
        my_list.remove(sentence)
    # if '' in my_list:
      # my_list.remove('')
    simil_list = []
    for sent in my_list:
      simil = util.cos_sim(base_embedding[0][0], pipe(sent)[0][0])
      simil_list.append(simil)
    if len(my_list) >=2:
      if simil_list[0] < simil_list[1]:
        empathy_utterances.append(my_list[0])
        base_utterances.append(my_list[1])
      else:
        empathy_utterances.append(my_list[1])
        base_utterances.append(my_list[0])
    elif len(my_list)==1:
      base_utterances.append(my_list[0])
  print(f"length of base utteranced {len(base_utterances)}")
  print(f"length of corpus {len(corpus)}")
  return base_utterances, empathy_utterances

In [19]:
# complete "split_utterances" dictionary with initial/additional base and empathy sentences
for emotion in emotions:
  emotion_filtered = ds.filter(lambda x: x['emotion'] == emotion)
  internal_dict = dict()
  if emotion == "joy":
    additional_questions = 'joy_base_sentences'
    base_question = "joy_question"
    columns = joy_question_column
    empathy_sentences = 'joy_empathy_sentences'
  else:
    additional_questions = 'base_sentences'
    base_question = "base_questions"
    columns = additional_questions_columns
    empathy_sentences = "empathy_sentences"


  internal_dict.setdefault(additional_questions,[])
  internal_dict.setdefault(empathy_sentences,[])
  for index, column in enumerate(columns):
    print(columns)
    corpus = emotion_filtered['train'][column]
    base_utterance, empathy_utterance = split_emp_base(corpus, base_utterance=split_utterances[base_question][index])
    internal_dict[additional_questions].append(base_utterance)
    internal_dict[empathy_sentences].extend(empathy_utterance)
  split_utterances[emotion] = internal_dict
  
  internal_dict_initial = dict()
  internal_dict_initial.setdefault('initial_base_sentences',[])
  internal_dict_initial.setdefault('initial_empathy_sentences',[])
  for index, column in enumerate(initial_questions_columns):
    corpus = emotion_filtered['train'][column]
    base_utterance, empathy_utterance = split_emp_base(corpus, base_utterance=split_utterances["initial_base_questions"][index])
    print(f"base_utterance{base_utterance}, empathy utterance {empathy_utterance}")
    internal_dict_initial['initial_base_sentences'].append(base_utterance)
    internal_dict_initial['initial_empathy_sentences'].append(empathy_utterance)
  
  internal_dict.update(internal_dict_initial)
  split_utterances[emotion] = internal_dict


# print(split_utterances)

  0%|          | 0/1 [00:00<?, ?ba/s]

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
length of base utteranced 50
length of corpus 50
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wro

  0%|          | 0/1 [00:00<?, ?ba/s]

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
length of base utteranced 50
length of corpus 50
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wro

  0%|          | 0/1 [00:00<?, ?ba/s]

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?']
length of base utteranced 50
length of corpus 50
['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wro

  0%|          | 0/1 [00:00<?, ?ba/s]

['Would you like to attempt protocols for joy?']
length of base utteranced 50
length of corpus 50
length of base utteranced 0
length of corpus 50
base_utterance[], empathy utterance []
length of base utteranced 0
length of corpus 50
base_utterance[], empathy utterance []
length of base utteranced 0
length of corpus 50
base_utterance[], empathy utterance []
length of base utteranced 0
length of corpus 50
base_utterance[], empathy utterance []
length of base utteranced 0
length of corpus 50
base_utterance[], empathy utterance []


In [20]:
len(split_utterances['joy']['joy_base_sentences'][0])
len(split_utterances['joy']['joy_empathy_sentences'][0])

48

In [21]:
split_utterances['anger']

{'base_sentences': [[' Возможно вы когда либо в последнее время испытывали чувства к кому-либо, как зависть, ненависть или злоба',
   'Испытывали ли вы когда  - либо, негативные чувства : зависть, ненависть или злость',
   'Итак: в последнее время вы были злы на кого-то ненавидели всей душой или завидовали',
   'Послушайте, возможно это действительно трудно описать, но скажите - может вы на кого-то зол\\зла, или что-то вызывает у вас чувство ненависти',
   ' У вас есть к кому-то чувство неприязни',
   'Вы испытываете кому-либо агрессию, злость, зависть',
   ' Вы испытывете резко отрицательные эмоции к кому-нибудь',
   'Зависть, ненависть, злость для вас знакомые чувства, вы их испытываете к кому-либо',
   ' Такие чувства как зависть, ненависть, злость испытывали ли вы к кому нибудь',
   'Возможно вы таите на кого-то негативные чувства',
   ' Вы испытываете к кому-либо : злость, презрение, ненависть, зависть',
   ' Вы испытывали зависть, ненависть или злость к кому нибудь',
   ' У вас в

In [22]:
split_utterances['anger']['initial_empathy_sentences'][0]

[' Ваши эмоции вызваны конкретным событием',
 'Благодарю вас за доверие, что обратились ко мне',
 'Не торопитесь и вспомните, что происходило с вами в последнее время',
 'Благодарю вас за то, что рассказали мне о своей проблеме',
 'Спасибо, что доверились мне',
 ' Правильно ли полагать, что чувство злости непосредственно связанно с каким-либо эпизодом из вашей жизни',
 'Вспомните, когда вас впервые охватило это чувство',
 'Сочувствую вам и надеюсь, мы сможем "погасить" эту злобу',
 ' Какой-то конкретный случай из жизни вызвал у вас такое состояние']

In [23]:
x = ds.filter(lambda x: x['emotion'] == "joy")['train']

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
x

Dataset({
    features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?'],
    num_rows: 50
})

### Inspecting split instances

In [25]:
# split_utterances['anger']['initial_empathy_sentences']
dict_df = dict()
dict_df_aq = dict()
dict_joy = dict()

for emotion in emotions:
  if emotion != "joy":
    dict_df[emotion] = dict()
    dict_df_aq[emotion] = dict()
    for index, question in enumerate(initial_questions_columns):
        question = initial_questions_columns[index]
        dict_df[emotion][question] = dict()
        dict_df[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion)['train'].num_rows
        # dict_df[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion).num_rows
        dict_df[emotion][question]['empathy'] =len(split_utterances[emotion]['initial_empathy_sentences'][index])
        dict_df[emotion][question]['base'] =len(split_utterances[emotion]['initial_base_sentences'][index]) 

    dict_df_aq[emotion] = dict()
    for index, question in enumerate(additional_questions_columns):
        question = additional_questions_columns[index]
        dict_df_aq[emotion][question] = dict()
        dict_df_aq[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion)['train'].num_rows
        # dict_df_aq[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion).num_rows
        dict_df_aq[emotion][question]['empathy'] =len(split_utterances[emotion]['empathy_sentences'])
        dict_df_aq[emotion][question]['base'] =len(split_utterances[emotion]['base_sentences'][index]) 

  else:
    dict_joy[emotion] = dict()
    for index, question in enumerate(joy_question_column):
      dict_joy[emotion][question] = dict()
      dict_joy[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion)['train'].num_rows
      # dict_joy[emotion][question]['original']= ds.filter(lambda x: x['emotion'] == emotion).num_rows
      dict_joy[emotion][question]['empathy'] =len(split_utterances[emotion]['joy_empathy_sentences'][index])
      dict_joy[emotion][question]['base'] =len(split_utterances[emotion]['joy_base_sentences'][index]) 

    # for index, question in enumerate(initial_questions_columns):
    #     question = initial_questions_columns[index]
    #     dict_df[emotion][question] = []
    #     dict_df[emotion][question].append(len(split_utterances[emotion]['initial_base_sentences'][index]))

In [26]:
print(ds.filter(lambda x: x['emotion'])['train'].num_rows)

  0%|          | 0/1 [00:00<?, ?ba/s]

200


In [27]:
pd.DataFrame.from_dict(dict_df)

,fear,sadness,anger
Was this emotion triggered by a specific event?,"{'original': 50, 'empathy': 16, 'base': 50}","{'original': 50, 'empathy': 21, 'base': 50}","{'original': 50, 'empathy': 9, 'base': 50}"
Was the event recent?,"{'original': 50, 'empathy': 8, 'base': 50}","{'original': 50, 'empathy': 1, 'base': 50}","{'original': 50, 'empathy': 5, 'base': 50}"
Did you find protocol 6 distressing?,"{'original': 50, 'empathy': 33, 'base': 50}","{'original': 50, 'empathy': 22, 'base': 50}","{'original': 50, 'empathy': 30, 'base': 50}"
Did you find protocol 11 distressing?,"{'original': 50, 'empathy': 35, 'base': 50}","{'original': 50, 'empathy': 22, 'base': 50}","{'original': 50, 'empathy': 31, 'base': 50}"
Is it ok to ask additional questions?,"{'original': 50, 'empathy': 9, 'base': 50}","{'original': 50, 'empathy': 9, 'base': 50}","{'original': 50, 'empathy': 2, 'base': 50}"


In [28]:
pd.DataFrame.from_dict(dict_df_aq)

,fear,sadness,anger
Have you strongly felt or expressed any of the following emotions towards someone?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
Do you believe you should be the savior of someone else?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
Do you feel that you are trying to control someone?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
"Do you see yourself as the victim, blaming someone else for how negative you feel?","{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
Are you always blaming and accusing yourself for when something goes wrong?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"
Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,"{'original': 50, 'empathy': 168, 'base': 50}","{'original': 50, 'empathy': 152, 'base': 50}","{'original': 50, 'empathy': 147, 'base': 50}"


In [29]:
pd.DataFrame.from_dict(dict_joy)

,joy
Would you like to attempt protocols for joy?,"{'original': 50, 'empathy': 48, 'base': 50}"


## Cluster split empathy and base sentences. Generate sentences

In [30]:
def cluster(num_clusters, corpus):
  embedder = SentenceTransformer('all-MiniLM-L6-v2')
  corpus_embeddings = embedder.encode(corpus)

  # Perform kmean clustering
  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)
  cluster_assignment = clustering_model.labels_
  clustered_sentences = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
      clustered_sentences[cluster_id].append(corpus[sentence_id])
  for i, cluster in enumerate(clustered_sentences):
      print("Cluster ", i+1)
      print(cluster)
      print("")
  return clustered_sentences

## Generate utterances

In [31]:
def permute_sentences(corpus, num_clusters):
  base_utterances, empathy_utterances = split_emp_base(corpus, base_utterance="Ваше состояние связано с каким-либо конкретным событием?")
  base_clusters = cluster(num_clusters, base_utterances)
  empathy_clusters = cluster(num_clusters, empathy_utterances)
  i = 0
  permutations = []
  for cluster_group in base_clusters:
    for base_sentence in cluster_group:
      for _ in range(2):
        empathy_sentence = random.choice(empathy_clusters[(i - i//10*10)])
        synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
        i += 1
        permutations.append(synthetic_sentence)
  # downsized_base = select_one_from_cluster(base_clusters)
  # downsized_empathy = select_one_from_cluster(empathy_clusters)
  # permutations = permute(downsized_empathy, downsized_base)
  return permutations

In [32]:
df_dict = dict()
df_dict

{}

In [ ]:
# for emotion in emotions:
#   df_dict[emotion] = dict()
#   d = split_utterances[emotion]
#   # generate sentences for additional questions
#   if emotion == "sadness":
#     for index, base in enumerate(additional_questions_columns):
#       print(index, base)
#       # print(index, base)
#       base_utterances = d['base_sentences'][index]
#       empathy_utterances = d['empathy_sentences']
#       base_clusters = cluster(num_clusters=5, corpus=base_utterances)
#       empathy_clusters = cluster(num_clusters=5, corpus=empathy_utterances)
#       i = 0
#       permutations = []
#       for cluster_group in base_clusters:
#         for base_sentence in cluster_group:
#           for _ in range(2):
#             empathy_sentence = random.choice(empathy_clusters[(i - i//5*5)])
#             synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
#             i += 1
#             permutations.append(synthetic_sentence)
#       df_dict[emotion][base] = permutations
#   # generate sentences for initial questions
#     for index, base in enumerate(initial_questions_columns):
#       print(index, base)
#       # print(index, base)
#       base_utterances = d['initial_base_sentences'][index]
#       empathy_utterances = d['initial_empathy_sentences'][index]
#       base_clusters = cluster(num_clusters=10, corpus=base_utterances)
#       if len(empathy_utterances) <= 10:
#         empathy_clusters = cluster(num_clusters=3, corpus=empathy_utterances)
#         number_emp_cl = 3
#       else:
#         empathy_clusters = cluster(num_clusters=10, corpus=empathy_utterances)
#         number_emp_cl = 5
#       i = 0
#       permutations = []
#       for cluster_group in base_clusters:
#         for base_sentence in cluster_group:
#           # new
#           if len(base_sentence) <= 3:
#             pass
#           for _ in range(2):
#             while len(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])==0:
#               i += 1
#             empathy_sentence = random.choice(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])
#             synthetic_sentence = ".".join((empathy_sentence, base_sentence)) + "?"
#             i += 1
#             permutations.append(synthetic_sentence)
#       df_dict[emotion][base] = permutations


In [33]:
for emotion in emotions:
  print(emotion)
  df_dict[emotion] = dict()
  d = split_utterances[emotion]
  # generate sentences for additional questions
  if emotion != "joy":
    for index, base in enumerate(additional_questions_columns):
      print(index, base)
      # print(index, base)
      base_utterances = d['base_sentences'][index]
      empathy_utterances = d['empathy_sentences']

      base_clusters = cluster(num_clusters=5, corpus=base_utterances)
      
      number_emp_cl = min(len(empathy_utterances), 5)
      print(f"number of clusters {number_emp_cl}")
      if number_emp_cl == 0:
        empathy_clusters = []
      else:
        empathy_clusters = cluster(num_clusters=number_emp_cl, corpus=empathy_utterances)


      empathy_clusters = cluster(num_clusters=number_emp_cl, corpus=empathy_utterances)
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          # if len(base_sentence) <= 3 or len(empathy_clusters)==0:
          #   pass
          for _ in range(2):
            while len(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])==0:
              i += 1
            if number_emp_cl != 0 and len(base_sentence) > 5:
              empathy_sentence = random.choice(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])
              synthetic_sentence = ". ".join((empathy_sentence, base_sentence)) + "?"
              i += 1
              permutations.append(synthetic_sentence)
          permutations.append(base_sentence)
      df_dict[emotion][base] = permutations
  # generate sentences for initial questions
    for index, base in enumerate(initial_questions_columns):
      print(index, base)
      # print(index, base)
      base_utterances = d['initial_base_sentences'][index]
      empathy_utterances = d['initial_empathy_sentences'][index]
      base_clusters = cluster(num_clusters=5, corpus=base_utterances)

      number_emp_cl = min(len(empathy_utterances), 5)
      print(f"number of clusters {number_emp_cl}")
      if number_emp_cl == 0:
        empathy_clusters = []
      else:
        empathy_clusters = cluster(num_clusters=number_emp_cl, corpus=empathy_utterances)
      # if len(empathy_utterances) <= 10:
      #   empathy_clusters = cluster(num_clusters=3, corpus=empathy_utterances)
      #   number_emp_cl = 3
      # else:
      #   empathy_clusters = cluster(num_clusters=10, corpus=empathy_utterances)
      #   number_emp_cl = 5
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          # new
          # if len(base_sentence) <= 3 or len(empathy_clusters)==0:
          #   pass
          for _ in range(2):
            while len(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])==0:
              i += 1
            if len(base_sentence) >= 4:
              empathy_sentence = random.choice(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])
              synthetic_sentence = ". ".join((empathy_sentence, base_sentence)) + "?"
              i += 1
              permutations.append(synthetic_sentence)
          permutations.append(base_sentence)
      df_dict[emotion][base] = permutations
  else:
    for index, base in enumerate(joy_question_column):
      print(index, base)
      base_utterances = d['joy_base_sentences'][index]
      empathy_utterances = d['joy_empathy_sentences']
      base_clusters = cluster(num_clusters=10, corpus=base_utterances)
      print(empathy_utterances)
      empathy_clusters = cluster(num_clusters=5, corpus=empathy_utterances)
      i = 0
      permutations = []
      for cluster_group in base_clusters:
        for base_sentence in cluster_group:
          # new
          # if len(base_sentence) <= 3:
          #   pass
          for _ in range(2):
            while len(empathy_clusters[(i - i//5*5)])==0:
              i += 1
            if len(base_sentence) >= 4:
              empathy_sentence = random.choice(empathy_clusters[(i - i//5*5)])
              synthetic_sentence = ". ".join((empathy_sentence, base_sentence)) + "?"
              i += 1
              permutations.append(synthetic_sentence)
          permutations.append(base_sentence)
      df_dict[emotion][base] = permutations
      # print(df_dict[emotion][base])

  

      # print(df_dict[emotion][base])

  
  # print(df_dict[emotion])
  



fear
0 Have you strongly felt or expressed any of the following emotions towards someone?


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Cluster  1
['Испытываете ли к кому-нибудь негативные чувства, может быть есть раздражающий человек', ' У вас есть к кому-то чувство неприязни', 'Вы чувствуете неприязнь к кому-либо', 'Вы чувствуете неприязнь к кому-либо', 'В данный момент вы испытываете резко отрицательные чувства к кому-либо', 'Вы чувствуете неприязнь к кому-либо']

Cluster  2
['Как вы считаете испытываете ли  вы к кому нибудь злость', ' Вас беспокоили какие-либо из этих чувств: зависть, ненависть, злость', ' Испытывали ли вы к кому либо зависть, ненависть или злость', ' Скажите, вы когда-либо испытывали такие чувства, как зависть, ненависть, злость', ' Возможно вы испытывали какие-то чувства из следующих ненависть, злость или зависть', ' Вы испытывали когда - либо такие чувства как зависть, ненависть или злость', ' Вы испытываете злость по отношению к кому-то, может быть, даже ненависть', ' Вы ранее испытывали ненависть, зависть или злобу по отношению к кому-нибудь', 'Скажите, вы испытывали такие чувства: зависть, зл

In [ ]:
# emotion = "anger"

# df_dict[emotion] = dict()
# d = split_utterances[emotion]
# # generate sentences for additional questions
# if emotion != "joy":

# # generate sentences for initial questions
#   for index, base in enumerate(initial_questions_columns):
#     print(index, base)
#     # print(index, base)
#     base_utterances = d['initial_base_sentences'][index]
#     empathy_utterances = d['initial_empathy_sentences'][index]
#     base_clusters = cluster(num_clusters=5, corpus=base_utterances)

#     number_emp_cl = min(len(empathy_utterances), 5)
#     print(f"number of clusters {number_emp_cl}")
#     if number_emp_cl == 0:
#       empathy_clusters = []
#     else:
#       empathy_clusters = cluster(num_clusters=number_emp_cl, corpus=empathy_utterances)
#     i = 0
#     permutations = []
#     for cluster_group in base_clusters:
#       for base_sentence in cluster_group:
#         # new
#         # if len(base_sentence) <= 3 or len(empathy_clusters)==0:
#         #   pass
#         for _ in range(2):
#           while len(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])==0:
#             i += 1
#           if len(base_sentence) >= 4:
#             empathy_sentence = random.choice(empathy_clusters[(i - i//number_emp_cl*number_emp_cl)])
#             synthetic_sentence = ". ".join((empathy_sentence, base_sentence)) + "?"
#             i += 1
#             permutations.append(synthetic_sentence)
#         permutations.append(base_sentence)
#     df_dict[emotion][base] = permutations
# else:
#   for index, base in enumerate(joy_question_column):
#     print(index, base)
#     base_utterances = d['joy_base_sentences'][index]
#     empathy_utterances = d['joy_empathy_sentences']
#     base_clusters = cluster(num_clusters=10, corpus=base_utterances)
#     print(empathy_utterances)
#     empathy_clusters = cluster(num_clusters=5, corpus=empathy_utterances)
#     i = 0
#     permutations = []
#     for cluster_group in base_clusters:
#       for base_sentence in cluster_group:
#         # new
#         # if len(base_sentence) <= 3:
#         #   pass
#         for _ in range(2):
#           while len(empathy_clusters[(i - i//5*5)])==0:
#             i += 1
#           if len(base_sentence) >= 4:
#             empathy_sentence = random.choice(empathy_clusters[(i - i//5*5)])
#             synthetic_sentence = ". ".join((empathy_sentence, base_sentence)) + "?"
#             i += 1
#             permutations.append(synthetic_sentence)
#         permutations.append(base_sentence)
#     df_dict[emotion][base] = permutations

In [ ]:
# equalise a length of utterances per emotion
def make_equal_length(dictionary):
  min_rows = min([len(dictionary[base]) for base in dictionary])
  for key in dictionary:
    dictionary[key] = dictionary.get(key)[:min_rows]
  return dictionary

In [34]:
# build a dataset with synthetic sentences
synthetic_df_columns = additional_questions_columns+initial_questions_columns+['emotion']
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)

for emotion in df_dict:
  # print(df_dict[emotion])
  # emotion_dict = make_equal_length(df_dict[emotion])
  # df = pd.DataFrame.from_dict(emotion_dict)
  # df = pd.DataFrame.from_dict(df_dict[emotion])

  df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df_dict[emotion].items() ]))
  df['emotion'] = [emotion for i in range(len(df))]
  synthetic_df = pd.concat([synthetic_df, df])

In [35]:
# check
synthetic_df[synthetic_df['emotion']=='sadness']

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,Is it ok to ask additional questions?,emotion,Would you like to attempt protocols for joy?
0,"Я хочу задать вам один не простой вопрос, пост...","Благодарю за то, что вы со мной поделились. В...","Я хочу задать вам один не простой вопрос, пост...","Я могу представить, что следующий вопрос может...",Благодарю заранее за честный ответ. Поделитес...,Благодарю заранее за честный ответ. Возникают...,Я благодарен вам за оказанное доверие. Я хоте...,Мы обязательно постараемся прогнать это чувств...,"Извините, если вам тяжело вспоминать об этом о...","Спасибо, что доверились мне. Вы почувствовали...","Я сожалею, что вам сейчас плохо. Скажите, пока...",Вы очень открытый собеседник ☺️. Чтобы понять ...,sadness,NaN
1,"Вы открыты со мной, и я вам благодарен. Вы ко...","Попытайтесь ответить на мой вопрос, пожалуйста...","Конечно, я знаю, что вопрос не самый простой. ...","Спасибо вам большое, что были со мной откровен...","Очень хорошо, спасибо за вашу откровенность и ...","Я понимаю вас, вопрос достаточно тяжёлый. Воз...","Я очень рад, что вы настолько открыры, я. Я х...","Искренне вам сочувствую, и сделаю все, чтобы п...","Извините, если вам тяжело вспоминать об этом о...",Поделитесь вашими ощущениями о методики 6: вы...,"Сочувствую вам 🥲. Скажите, показалась ли вам м...","Я, надеюсь, я вас несильно утомил. Чтобы понят...",sadness,NaN
2,"Вы кого-нибудь ненавидите, завидуете, злитесь","Вам иногда кажется, что вы ставите цель опека...","Вам когда-нибудь казалось, что вы пытаетесь ко...","Бывает, что вы чувствуете себя заложником (зал...","Поделитесь, часто ли бывает ли так, что вы ви...","Возникают ли у вас ситуации, когда вы доказыв...",Я хотел бы узнать еще кое-что,"Скажите, ваше душевное расстройство обусловле...",Это было не так давно,Вы почувствовали после методики 6 сильную душ...,"Скажите, показалась ли вам методика 11 неприятной","Чтобы понять ситуацию, я хочу получить ответы ...",sadness,NaN
3,Благодарю вас за искренность. Вы испытывали в ...,"Спасибо , что поделились. Вы ощущаете себя спа...","Спасибо за доверие. Вам не кажется, что вы пыт...","Извините, вопрос не из лёгких. Бывает, что вы ...","Хотелось бы задать вам непростой вопрос, на ко...","Очень хорошо, спасибо за вашу откровенность и ...",Спасибо что доверились. Еще хотел бы уточнить...,"Спасибо, что делитесь. Скажите, ваше состояни...","Извините, если вам тяжело вспоминать об этом о...","Благодарю, что вы решили со мной поделиться. П...","Мне приятно что вы открылись мне. Скажите, пож...","Приятно, что вы доверились мне. Возможно мы вм...",sadness,NaN
4,"Догадываюсь, это сложный вопрос. Вы испытывали...",Я благодарен вам за вашу открытость. Вы ощущае...,"Я осознаю, что это не самый лёгкий вопрос. Вам...","Благодарю за доверие, милый друг. Бывает, что ...","Наверное, это не самый приятный вопрос, но все...",У меня возник ещё один нелёгкий вопрос. К сожа...,"Я понимаю, что это не самый лёгкий вопрос. Ещ...","Благодарю. Скажите, ваше состояние возникло /...","Извините, если вам тяжело вспоминать об этом о...",Спасибо за откровенный ответ. Практическое пр...,"Благодарю за откровенность. Скажите, пожалуйст...",Я очень хочу вам помочь. Возможно мы вместе см...,sadness,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Я вас благодарю за вашу откровенность. О

In [36]:
synthetic_df[synthetic_df['emotion']=='anger']['Is it ok to ask additional questions?']

0      Спасибо за откровенность. Хотелось бы у вас сп...
1      Мне нужно увидеть ситуацию со всех углов. Хоте...
2      Хотелось бы у вас спросить пару вопросов, чтоб...
3      Спасибо за откровенность. Мне бы хотелось зада...
4      Мне нужно увидеть ситуацию со всех углов. Мне ...
                             ...                        
145    Мне нужно увидеть ситуацию со всех углов. Я хо...
146    Я хочу задать вам несколько наводящих вопросов...
147    Спасибо за откровенность. Вы не возражаете, ес...
148    Мне нужно увидеть ситуацию со всех углов. Вы н...
149    Вы не возражаете, если я задам несколько вопро...
Name: Is it ok to ask additional questions?, Length: 150, dtype: object

In [37]:
d

{'joy_base_sentences': [[' У меня есть новое интересное задание для вас, чтобы поддержать положительную динамику, вы не против',
   ' Чтобы закрепить ваш бодрый настрой, могу посоветовать некоторые упражнения',
   ' Думаю вам очень понравится моё предложение по эффективной программе - начнём',
   ' Я бы хотел предложить вам, одну интересную методику - вы не потив',
   ' Я вам порекомендую несколько упражнений на выбор для фиксации вашего настроения, если вы не против',
   ' Могу предложить вам упражнение, чтобы стало ещё лучше, не против',
   ' Я бы хотела показать вам одно упражнение для поднятия настроения, чтобы у вас и дальше все было хорошо, и вы чувствовали себя прекрасно',
   '  Чтобы закрепить ваш бодрый настрой, могу посоветовать некоторые упражнения',
   ' Хочу посоветовать вам рекомендацию по выполнению неких упражнений - согласны',
   ' Желаете попробовать упражнения для сохранения положительной динамики',
   '  Я бы хотел вам посоветовать выбрать из ряда упражений для подд

# Convert pandas into a Huggingface dataset

In [38]:
dataset = Dataset.from_pandas(synthetic_df)

In [39]:
dataset = dataset.remove_columns('__index_level_0__')

In [40]:
# build a dataset with synthetic sentences
synthetic_df_columns = additional_questions_columns+initial_questions_columns+['emotion']
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)

for emotion in df_dict:
  # print(df_dict[emotion])
  # emotion_dict = make_equal_length(df_dict[emotion])
  # df = pd.DataFrame.from_dict(emotion_dict)
  # df = pd.DataFrame.from_dict(df_dict[emotion])

  df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df_dict[emotion].items() ]))
  df['emotion'] = [emotion for i in range(len(df))]
  synthetic_df = pd.concat([synthetic_df, df])

In [41]:
dataset.set_format(type="pandas")
emotions = dataset.unique('emotion')
dataset.reset_format()

# Save a dataset

In [42]:
dataset.save_to_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

In [43]:
# check that it loaded correctly
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

In [44]:
ds

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?'],
    num_rows: 600
})

In [45]:
ds.set_format(type="pandas")


In [46]:
len(ds.filter(lambda example: example["emotion"]=="joy"))

  0%|          | 0/1 [00:00<?, ?ba/s]

150

# Example of generating synthetic dataset

In [ ]:
from numpy.random.mtrand import randint
synthetic_dict = dict()
for emotion in emotions:
  if emotion != "joy":
    subset = dataset.filter(lambda row: row['emotion']==emotion)
    subset = subset.remove_columns(["emotion",'Would you like to attempt protocols for joy?'] )
    internal_dict = dict()
    max_int = len(subset)
    columns = subset.column_names
    for column_name in columns:
      internal_dict[column_name] = []
      i = 0
      while i < 10:
        utterance = subset[column_name][random.randint(0, max_int)]
        if not pd.isna(utterance):
          i += 1
          internal_dict[column_name].append(utterance)
      
      synthetic_dict[emotion] = internal_dict
  else:
    subset = dataset.filter(lambda row: row['emotion']==emotion)
    subset = subset.remove_columns(['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion',] )
    internal_dict = dict()
    max_int = len(subset)
    columns = subset.column_names
    for column_name in columns:
      internal_dict[column_name] = []
      i = 0
      while i < 10:
        utterance = subset[column_name][random.randint(0, max_int)]
        if not pd.isna(utterance):
          i += 1
          internal_dict[column_name].append(utterance)
      
      synthetic_dict[emotion] = internal_dict

        


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
emotion = "anger"
subsetDataSet = dataset.filter(lambda row: row['emotion']==emotion)
if emotion == "joy":
    subsetDataSet = subsetDataSet.remove_columns(['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)
else:
    subsetDataSet = subsetDataSet.remove_columns(['Would you like to attempt protocols for joy?'] )
    internal_dict = dict()
    max_int = len(subsetDataSet)
    columns = subsetDataSet.column_names
    for column_name in columns:
        internal_dict[column_name] = []
        i = 0
        while i < 10:
            utterance = subsetDataSet[column_name][random.randint(0, max_int)]
            if not pd.isna(utterance):
                i += 1
                internal_dict[column_name].append(utterance)

synthetic_df_columns = subsetDataSet.column_names
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)
df = pd.DataFrame.from_dict(internal_dict)
synthetic_df = pd.concat([synthetic_df, df])
dataset = Dataset.from_pandas(synthetic_df)
    

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', '__index_level_0__'],
    num_rows: 10
})

In [ ]:
# build a dataset with synthetic sentences
synthetic_df_columns = additional_questions_columns+initial_questions_columns+['emotion']
synthetic_df = pd.DataFrame(columns=synthetic_df_columns)

for emotion in synthetic_dict:
  # print(df_dict[emotion])
  # emotion_dict = make_equal_length(df_dict[emotion])
  df = pd.DataFrame.from_dict(synthetic_dict)
  df = pd.DataFrame.from_dict(synthetic_dict[emotion])
  df['emotion'] = [emotion for i in range(len(df))]
  synthetic_df = pd.concat([synthetic_df, df])

In [ ]:
synthetic_df
dataset = Dataset.from_pandas(synthetic_df)